In [1]:
import os
print(os.environ["MINIO_ACCESS_KEY"])

mEk4ek2HCmeb3xCK2twM


In [2]:
from pyspark.sql import SparkSession

In [ ]:
spark = (
        SparkSession.builder.master("spark://spark-master:7077")
        .appName("stg_clockify__time_entries")
        .config(
            "spark.jars.packages",
            "org.apache.hadoop:hadoop-aws:3.3.4,org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.4.2",
        )
        .config("spark.hadoop.fs.s3a.access.key", os.environ["MINIO_ACCESS_KEY"])
        .config("spark.hadoop.fs.s3a.secret.key", os.environ["MINIO_SECRET_KEY"])
        .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
        .config("spark.hadoop.fs.s3a.path.style.access", "true")
        .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
        .config(
            "spark.sql.catalog.clockify_catalog",
            "org.apache.iceberg.spark.SparkCatalog",
        )
        .config("spark.sql.catalog.clockify_catalog.type", "hadoop")
        .config(
            "spark.sql.catalog.clockify_catalog.warehouse", f"s3a://lakehouse/iceberg/"
        )
        .getOrCreate()
    )

In [4]:
df = spark.read.format("iceberg").load("clockify_catalog.bronze.time_entries")

25/04/30 00:35:53 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [5]:
df.limit(5).toPandas().head()

,billable,costRate,customFieldValues,description,hourlyRate,id,isLocked,kioskId,projectId,tagIds,taskId,timeInterval,type,userId,workspaceId
0,False,"(0, R$)",[],AIRFLOW,"(0, R$)",681086462e56197b9fdc38a3,False,None,5e9f4704ea8094116e994d87,[],None,"(PT1H30M, 2025-04-28T19:30:00Z, 2025-04-28T18:...",REGULAR,5e95c064ea8094116e8e0a54,5e95c064ea8094116e8e0a56
1,True,"(0, R$)",[],FN,"(0, R$)",68108623685bcf6ccbc046a8,False,None,61e54e2ddc3256444ce00210,[],None,"(PT8H, 2025-04-28T16:00:00Z, 2025-04-28T08:00:...",REGULAR,5e95c064ea8094116e8e0a54,5e95c064ea8094116e8e0a56


In [10]:
spark.read.parquet("s3a://raw/clockify/time-entries/parquet/*.parquet").printSchema()

root
 |-- billable: boolean (nullable = true)
 |-- costRate: struct (nullable = true)
 |    |-- amount: long (nullable = true)
 |    |-- currency: string (nullable = true)
 |-- customFieldValues: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- description: string (nullable = true)
 |-- hourlyRate: struct (nullable = true)
 |    |-- amount: long (nullable = true)
 |    |-- currency: string (nullable = true)
 |-- id: string (nullable = true)
 |-- isLocked: boolean (nullable = true)
 |-- kioskId: string (nullable = true)
 |-- projectId: string (nullable = true)
 |-- tagIds: string (nullable = true)
 |-- taskId: string (nullable = true)
 |-- timeInterval: struct (nullable = true)
 |    |-- duration: string (nullable = true)
 |    |-- end: string (nullable = true)
 |    |-- start: string (nullable = true)
 |-- type: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- workspaceId: string (nullable = true)



In [11]:
spark.read.json("s3a://raw/clockify/time-entries/*.json").printSchema()

root
 |-- billable: boolean (nullable = true)
 |-- costRate: struct (nullable = true)
 |    |-- amount: long (nullable = true)
 |    |-- currency: string (nullable = true)
 |-- customFieldValues: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- description: string (nullable = true)
 |-- hourlyRate: struct (nullable = true)
 |    |-- amount: long (nullable = true)
 |    |-- currency: string (nullable = true)
 |-- id: string (nullable = true)
 |-- isLocked: boolean (nullable = true)
 |-- kioskId: string (nullable = true)
 |-- projectId: string (nullable = true)
 |-- tagIds: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- taskId: string (nullable = true)
 |-- timeInterval: struct (nullable = true)
 |    |-- duration: string (nullable = true)
 |    |-- end: string (nullable = true)
 |    |-- start: string (nullable = true)
 |-- type: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- workspaceId: string (nullable = t

25/04/30 00:49:47 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
25/04/30 00:49:47 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:291)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:981)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:165)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:263)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:170)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.proce

In [6]:
spark.read.format("iceberg").load("s3a://staging/bronze/time_entries").show()

25/04/30 00:36:02 WARN HadoopTableOperations: Error trying to recover version-hint.txt data for s3a://staging/bronze/time_entries/metadata/version-hint.text
java.io.FileNotFoundException: No such file or directory: s3a://staging/bronze/time_entries/metadata/version-hint.text
	at org.apache.hadoop.fs.s3a.S3AFileSystem.s3GetFileStatus(S3AFileSystem.java:3866)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.innerGetFileStatus(S3AFileSystem.java:3688)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.extractOrFetchSimpleFileStatus(S3AFileSystem.java:5401)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.open(S3AFileSystem.java:1465)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.open(S3AFileSystem.java:1441)
	at org.apache.hadoop.fs.FileSystem.open(FileSystem.java:976)
	at org.apache.iceberg.hadoop.HadoopTableOperations.findVersion(HadoopTableOperations.java:318)
	at org.apache.iceberg.hadoop.HadoopTableOperations.refresh(HadoopTableOperations.java:104)
	at org.apache.iceberg.hadoop.HadoopTableOperations.curren

Py4JJavaError: An error occurred while calling o57.load.
: org.apache.iceberg.exceptions.RuntimeIOException: Failed to refresh the table
	at org.apache.iceberg.hadoop.HadoopTableOperations.refresh(HadoopTableOperations.java:126)
	at org.apache.iceberg.hadoop.HadoopTableOperations.current(HadoopTableOperations.java:84)
	at org.apache.iceberg.hadoop.HadoopTables.load(HadoopTables.java:94)
	at org.apache.iceberg.spark.SparkCatalog.loadFromPathIdentifier(SparkCatalog.java:772)
	at org.apache.iceberg.spark.SparkCatalog.load(SparkCatalog.java:639)
	at org.apache.iceberg.spark.SparkCatalog.loadTable(SparkCatalog.java:159)
	at org.apache.spark.sql.connector.catalog.CatalogV2Util$.getTable(CatalogV2Util.scala:363)
	at org.apache.spark.sql.execution.datasources.v2.DataSourceV2Utils$.loadV2Source(DataSourceV2Utils.scala:137)
	at org.apache.spark.sql.DataFrameReader.$anonfun$load$1(DataFrameReader.scala:210)
	at scala.Option.flatMap(Option.scala:271)
	at org.apache.spark.sql.DataFrameReader.load(D

In [ ]:
# spark.sql("SHOW NAMESPACES IN clockify_catalog").show()  # Lists all databases (namespaces)
spark.sql("SHOW TABLES IN clockify_catalog.bronze").show()  # Lists all tables in bronze

In [ ]:
sp